In [56]:
import pandas as pd

In [ ]:
# put products to correspond brands in a dictionary
df2 = pd.read_excel("cleaned_Dogfood.xlsx")
products = set(df2.product_name)
brands = set(df2.product_brand)
brand_products = {}
for brand in brands:
    brand_products[brand] = []
for product in products:
    clip = product.split()[0:2]
    clips = " ".join(clip)
    for brand in brands:
        if brand in clips:
            brand_products[brand].append(product)
            break
brand_products

In [200]:
# all attributes to get facters
def get_catergories(data, category): # input: "str"
        # return a list of catergories
        return list(set(data[category]))
    
def get_sns_rank(data, col): # input: data = DataFrame, col = list["str"..]
        # return a list of the order of sns avg group by catergories increasing order
        rank = data.groupby(col).sns.mean().sort_values().index
        return list(rank)

def get_market_percentage(data, col):# input: data = DataFrame, col = list["str"..]
        # return the market values sum of each catergories
        return data.groupby(col).total_price.sum()

def get_proportions(data, col):# input: data = DataFrame, col = list["str"..]
        # return the proportions of given input (1 or 2)
        if len(col) == 1:
                return data.groupby(col).size() / len(data)
        # for 2 inputs, the order decide which is the main(1st) and sub(2nd) category
        return data.groupby(col).size().groupby(level=0, group_keys=False).apply(lambda x: x/x.sum())

In [201]:
df2 = pd.read_excel("cleaned_Dogfood.xlsx")

price_level  income           
cheap        $100,000 or more     0.286563
             $20,000 - $39,999    0.160420
             $40,000 - $59,999    0.152194
             $60,000 - $79,999    0.166362
             $80,000 - $99,999    0.167733
             Less than $20,000    0.066728
good         $100,000 or more     0.282018
             $20,000 - $39,999    0.160965
             $40,000 - $59,999    0.170614
             $60,000 - $79,999    0.157895
             $80,000 - $99,999    0.167105
             Less than $20,000    0.061404
middle       $100,000 or more     0.307971
             $20,000 - $39,999    0.133023
             $40,000 - $59,999    0.157867
             $60,000 - $79,999    0.151656
             $80,000 - $99,999    0.182712
             Less than $20,000    0.066770
premium      $100,000 or more     0.316279
             $20,000 - $39,999    0.162791
             $40,000 - $59,999    0.145116
             $60,000 - $79,999    0.161395
             $80,000 - 

In [81]:
#demo demo-geographic helper function
def dg_validate(demogeo):
    # get input info of gender, age, state
    # output the column name needed
    column = []
    for gas in demogeo:
        if gas in {'M', 'F'}:
            column.append("gender")
            continue
        elif gas in {'55-64', '35-44', '25-34', '18-24', '65+', '45-54'}:
            column.append("age")
            continue
        elif gas in {'UT', 'AL', 'OR', 'MO', 'MT', 'WY', 'MN', 'SD', 'PA', 'CO', 'OK',
                   'HI', 'WA', 'SC', 'IL', 'WV', 'MI', 'WI', 'CA', 'MS', 'KS', 'ID',
                   'OH', 'IN', 'NM', 'VA', 'GA', 'LA', 'DC', 'ND', 'NY', 'MD', 'AK',
                   'DE', 'NV', 'AZ', 'IA', 'NC', 'NE', 'AR', 'FL', 'TX', 'KY', 'TN'}:
            column.append("st")
            continue
        else: # input validation
            raise ValueError("Invalid Args Input")
    return column

In [168]:
#demo pyschographic helper function
def pyscho_validate(pyscho):
    column = []
    # get input info of income target and price
    # output the column name needed
    for pi in pyscho:    
        if pi in {'$20,000 - $39,999', '$40,000 - $59,999', '$60,000 - $79,999',
                  '$80,000 - $99,999', 'Less than $20,000', '$100,000 or more'}:
            column.append("income")
        elif pi in {'middle', 'cheap', 'good', 'premium'}:
            column.append("price_level")
        else: # input validation
            raise ValueError("Invalid Args Input")
    return column

In [207]:
def woof(data, brand, demo_geo = None, ads_type = None, pyscho = None):
        # calculate the competebility of brand within all brands
        woof_score = 0
        brand_score = get_sns_rank(data, "product_brand").index(brand) / len(get_catergories(data, "product_brand"))
        brand_market = get_market_percentage(data, "product_brand")[brand]
        # calculate the demo-geographic impact score
        if demo_geo:
                demo_geo_score = get_market_percentage(data, dg_validate(demo_geo))
                for dg in demo_geo:
                        demo_geo_score = demo_geo_score.loc[dg]
                demo_geo_score = demo_geo_score/brand_market
                woof_score += demo_geo_score
        # calculate the behavioral impact score
        
        # calculate the pyschographic impact score
        if pyscho:
                pyscho_score = get_proportions(data, pyscho_validate(pyscho))
                for p in pyscho:
                        pyscho_score = pyscho_score.loc[p]
                woof_score += pyscho_score
        woof_score += brand_score
        return woof_score

In [209]:
woof(df2, "Alpha", demo_geo=["F", '18-24', "FL"], pyscho=['middle', 'Less than $20,000'])

0.5104732792842366